# Generate new annotations when the original went to a URL proxied through the UNC library's proxy

Unfortunately, I don't know that there is any way to update the existing annotations except by the person who made them in the first place...

You probably don't need to use this-- it is here to clean up some older example data, and I'm including it in the repo as an example.

**Note**: In order to use this, you have to have a hypothes.is API token with permissions to access the relevant hypothes.is group. It should be included in a file called "token" as a single line.

In [ ]:
import requests
import yaml
import os
import re
import json

In [ ]:
token = open('token').read().rstrip()
api_endpoint = 'https://hypothes.is/api'
group = 'DRL6xW1v'

proxy_host = '.libproxy.lib.unc.edu'

fields_to_keep = 'uri text tags group permissions target document references'.split()

In [ ]:
def search_for_annotations(querydata, token=token):
    '''Fetches, using the already-defined token and handling pagination'''
    # FIXME: search_after is described as more efficient in the API docs, but I don't know if it handles timestamp collisions
    fetched = []
    while True:
        response = requests.post(api_endpoint + '/search',
                            data=querydata,
                            headers={'Authorization': 'Bearer %s'%token}).json()
        fetched += response['rows']
        if response['total'] == len(fetched):
            return fetched
        querydata['offset'] = len(fetched)

In [ ]:
annotations_to_replace = search_for_annotations({'group': group, 'uri.parts': proxy_host})

In [ ]:
def remove_unc_libproxy_from_url(url):
    spliturl = url.split(proxy_host, 1)
    if len(spliturl) == 1:
        return spliturl[0]
    return spliturl[0].replace('-', '.') + spliturl[1]

In [ ]:
def fix_proxied_annotation(old_annotation):
    ann = {k: old_annotation[k] for k in fields_to_keep if k in old_annotation}
    ann['uri'] = remove_unc_libproxy_from_url(ann['uri'])
    for t in ann['target']:
        if proxy_host in t['source']:
            t['source'] = remove_unc_libproxy_from_url(t['source'])
    ann.setdefault('references', list()).append(old_annotation['id'])
    return ann

In [ ]:
for a in annotations_to_replace:
    post_response = requests.post(api_endpoint + '/annotations',
              json = fix_proxied_annotation(a),
              headers={'Authorization': 'Bearer %s'%token})
    print(post_response)
    print(post_response.json()['id'])